In [2]:
## importing libraries
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
import os,re
os.chdir("E:/pythontraining/imarticus/ML_python/naive_bayes/")

In [40]:
## tweets data
tweets=pd.read_csv("data/tweets.csv")

In [41]:
tweets=tweets[['airline_sentiment','text']]

In [42]:
tweets.isnull().sum()

airline_sentiment    0
text                 0
dtype: int64

In [43]:
tweets['airline_sentiment'].value_counts()

negative    9178
neutral     3099
positive    2363
Name: airline_sentiment, dtype: int64

In [49]:
tweets2=tweets[~tweets['airline_sentiment'].isin(['neutral'])].reset_index(drop=True)

In [50]:
tweets2['airline_sentiment'].value_counts()

negative    9178
positive    2363
Name: airline_sentiment, dtype: int64

In [51]:
def tweet_cleaner(tweet): 
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t\\'s]) |(\w+:\/\/\S+)", " ", tweet).split()) 

In [56]:
def tweet_clean1(x):
    w=re.sub("\'s"," is",tweet_cleaner(x))
    w=re.sub("\'","",w)
    return w

In [217]:
tweets['text'][3]

'@VirginAmerica it\'s really aggressive to blast obnoxious "entertainment" in your guests\' faces &amp; they have little recourse'

In [57]:
tweet_clean1(tweets['text'][3])

'it is really aggressive to blast obnoxious "entertainment in your guests faces &amp they have little recourse'

In [59]:
tweets2['clean_text']=tweets2['text'].apply(lambda x : tweet_clean1(x))

In [61]:
tweets2.head()

,airline_sentiment,text,clean_text
0,positive,@VirginAmerica plus you've added commercials t...,plus youve added commercials to the experience...
1,negative,@VirginAmerica it's really aggressive to blast...,"it is really aggressive to blast obnoxious ""en..."
2,negative,@VirginAmerica and it's a really big bad thing...,and it is a really big bad thing about it
3,negative,@VirginAmerica seriously would pay $30 a fligh...,seriously would pay $30 a flight for seats tha...
4,positive,"@VirginAmerica yes, nearly every time I fly VX...",yes nearly every time I fly VX this “ear worm ...


In [ ]:
tweets2['airline_sentiment'].value_counts()

In [62]:
tweets2.loc[tweets2["airline_sentiment"] == 'positive', "airline_sentiment",] = 1
tweets2.loc[tweets2["airline_sentiment"] == 'negative', "airline_sentiment",] = 0

In [64]:
tweets2['airline_sentiment'].value_counts()

0    9178
1    2363
Name: airline_sentiment, dtype: int64

In [218]:
9178/2363

3.8840457046127805

In [232]:
X_train, X_test, y_train, y_test = train_test_split(tweets2["clean_text"], tweets2["airline_sentiment"], test_size = 0.2, random_state = 4,
                                    stratify=tweets2["airline_sentiment"],shuffle=True)

In [231]:
from collections import Counter
Counter(y_train)

Counter({0: 7319, 1: 1913})

In [222]:
7342/1890

3.8846560846560845

In [226]:
7333/1899

3.861506055818852

In [229]:
Counter(y_test)

Counter({0: 1815, 1: 494})

In [223]:
1836/473

3.8816067653276956

### Count vectorizer

In [149]:
cv = CountVectorizer()
X_traincv = cv.fit_transform(X_train.values)
X_testcv = cv.transform(X_test.values)

In [150]:
y_train = y_train.astype('int')
y_test = y_test.astype('int')

In [151]:
naive_bayes = MultinomialNB()

In [152]:
naive_bayes.fit(X_traincv, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [153]:
y_test_pred = naive_bayes.predict(X_testcv)

In [154]:
# Testing accuracy
naive_bayes.score(X_testcv, y_test)

0.9077522737115634

In [155]:
from sklearn.metrics import classification_report,f1_score,accuracy_score
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.91      0.98      0.94      1836
           1       0.90      0.62      0.73       473

   micro avg       0.91      0.91      0.91      2309
   macro avg       0.91      0.80      0.84      2309
weighted avg       0.91      0.91      0.90      2309



In [211]:
####  tfidf
tf_idf = TfidfVectorizer(ngram_range = (1, 1),max_df=.1,use_idf =True,stop_words='english')

In [212]:
TfidfVectorizer

sklearn.feature_extraction.text.TfidfVectorizer

In [213]:
X_traintfidf = tf_idf.fit_transform(X_train.values)
X_testtfidf = tf_idf.transform(X_test.values)
y_train = y_train.astype('int')
y_test = y_test.astype('int')

In [214]:
naive_bayes = MultinomialNB()
naive_bayes.fit(X_traintfidf, y_train)
y_test_pred_tfidf = naive_bayes.predict(X_testtfidf)

In [215]:
naive_bayes.score(X_testtfidf, y_test)

0.8462537895192724

In [216]:
print(classification_report(y_test,y_test_pred_tfidf))

              precision    recall  f1-score   support

           0       0.84      1.00      0.91      1836
           1       0.98      0.25      0.40       473

   micro avg       0.85      0.85      0.85      2309
   macro avg       0.91      0.63      0.66      2309
weighted avg       0.87      0.85      0.81      2309



In [237]:
dfnew=pd.DataFrame(X_test)
dfnew['actuallabel']=y_test
dfnew['predictedlabel']=y_test_pred

In [239]:
dfnew

,clean_text,actuallabel,predictedlabel
9753,I understand that but Im hoping I can rectify ...,0,0
6528,why 730 and not 645 as scheduled?,0,0
4629,suggestion shades on the windows so travelers ...,0,0
10454,keep trying to call to change reservation and ...,0,0
5592,Token $75.00 Goodwill Voucher for passengers s...,0,0
...,...,...,...
5423,well here we go,1,0
4879,Thank you!,1,1
11234,Haha I had a boarding pass for 12B was boardin...,1,0
4090,breaking my heart at BWI 9 hrs,0,0
